In [1]:
import numpy as np
import pandas as pd
import re
import nltk #for NLP
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('universal_tagset')
nltk.download('reuters')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/somethingcreative-7/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/somethingcreative-7/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/somethingcreative-7/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package reuters to
[nltk_data]     /Users/somethingcreative-7/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/somethingcreative-7/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

#### Extract SkillsFuture competency framework data

In [3]:
###load job description data
userdata = '/Users/somethingcreative-7/Documents/Tessas Internship/Phase 2 preso/Supplementary documents/JDoutput.xlsx'
userdata = pd.read_excel(userdata)
userdata['Job Role'] = userdata['Job Role'].apply(lambda x: x.lstrip(' '))
#userdata.head(5)

In [4]:
print(list(sorted(userdata['Job Role'])))

['Applications Support Engineer', 'Artificial Intelligence Applied Researcher', 'Artificial Intelligence/Machine Learning Engineer', 'Associate Applications Support Engineer', 'Associate Business Analyst', 'Associate Data Centre Operations Engineer', 'Associate Database Support Engineer', 'Associate Embedded Systems Engineer', 'Associate Infrastructure Engineer', 'Associate Infrastructure Support Engineer', 'Associate Network Engineer', 'Associate Operations Centre Support Engineer', 'Associate Radio Frequency Engineer', 'Associate Security Analyst', 'Associate Software Engineer', 'Associate Systems Support Engineer', 'Associate UI Designer', 'Associate UX Designer', 'Automation and Orchestration Engineer', 'Business Analyst/Artificial Intelligence Translator', 'Business Architect', 'Business Development Manager', 'Business Intelligence Director', 'Business Intelligence Manager', 'Channel Sales Manager', 'Chief Data Officer/Chief Artificial Intelligence Officer', 'Chief Information Off

## USER input target job (select from competency frameworks)

In [5]:
###user select from dropdown list
target_job= "Software Engineer"

In [6]:
target_job_row = userdata.loc[userdata['Job Role']==target_job]
target_job_df = pd.DataFrame(target_job_row)
list(target_job_df['Skills'])

['Applications Development-Level4,Computational Thinking Intermediate, Applications Integration-Level4,Business Environment Analysis-Level3, Problem Solving Intermediate, Applications Support and Enhancement-Level3, Lifelong Learning Intermediate, Budgeting-Level3, Communication Basic , Teamwork Intermediate, Business Needs Analysis-Level3, Business Negotiation-Level3, Business Requirements Mapping-Level3, Business Risk Management-Level3, Change Management-Level3, Configuration Tracking-Level3, Data Design-Level3, Database Administration-Level4, Emerging Technology Synthesis-Level4, Performance Management-Level4, Problem Management-Level3, Product Management-Level4, Project Management-Level4, Quality Standards-Level4,,Software Configuration-Level3, Software Design-Level4, Software Testing-Level3, Solution Architecture-Level4, Stakeholder Management-Level3, System Integration-Level3, Test Planning-Level3, User Interface Design-Level4']

In [7]:
##extract skills col required for job # start with technical skills?
skills_df=pd.DataFrame(target_job_df['Skills'])
##transpose and clean, only technical skills pulled out- contains("-Level[0-9]")
skills_df=skills_df.assign(Skills=skills_df['Skills'].str.split(",")).explode('Skills')
skills_df['Skills'] = skills_df['Skills'].apply(lambda x: x.lstrip(' '))
skills_df = skills_df[skills_df['Skills'].str.contains("-Level[0-9]")]
skills_df['skills_original']=skills_df['Skills']

In [8]:
##extract skill levels into new column
skills_df['expected_level']= skills_df['Skills']
##pull out numeric 'Level' value 
skills_df['expected_level']= skills_df['expected_level'].apply(lambda i: i.rsplit('-Level', 1)[-1])
##pull out Skill name without level
skills_df['Skills']= skills_df['skills_original'].apply(lambda i: i.rsplit("-Level",1)[0])

skills_df

,Skills,skills_original,expected_level
0,Applications Development,Applications Development-Level4,4
0,Applications Integration,Applications Integration-Level4,4
0,Business Environment Analysis,Business Environment Analysis-Level3,3
0,Applications Support and Enhancement,Applications Support and Enhancement-Level3,3
0,Budgeting,Budgeting-Level3,3
0,Business Needs Analysis,Business Needs Analysis-Level3,3
0,Business Negotiation,Business Negotiation-Level3,3
0,Business Requirements Mapping,Business Requirements Mapping-Level3,3
0,Business Risk Management,Business Risk Management-Level3,3
0,Change Management,Change Management-Level3,3


In [9]:
#prep df for users' inputs
skills_df['user_level']=skills_df['expected_level']
skills_df=skills_df.reset_index(level=None, drop=True, inplace=False)
skills_df

,Skills,skills_original,expected_level,user_level
0,Applications Development,Applications Development-Level4,4,4
1,Applications Integration,Applications Integration-Level4,4,4
2,Business Environment Analysis,Business Environment Analysis-Level3,3,3
3,Applications Support and Enhancement,Applications Support and Enhancement-Level3,3,3
4,Budgeting,Budgeting-Level3,3,3
5,Business Needs Analysis,Business Needs Analysis-Level3,3,3
6,Business Negotiation,Business Negotiation-Level3,3,3
7,Business Requirements Mapping,Business Requirements Mapping-Level3,3,3
8,Business Risk Management,Business Risk Management-Level3,3,3
9,Change Management,Change Management-Level3,3,3


# user input their own actual skill level

In [10]:
print(str(len(skills_df)) + ' skills found for '+ target_job)

27 skills found for Software Engineer


In [11]:
## iterate down skills list to get user input for each skill
##user's inputs should be based on competency framwork

for i in skills_df['Skills']:
    index= skills_df.index[skills_df['Skills'] == i]
    ###get user input
    text= "Enter your proficiency for " + str(i) + " (number from 0-6):"
    new = int(input(text))
    skills_df.at[index, 'user_level']= new
skills_df

KeyboardInterrupt: Interrupted by user

# NEW FOR EVAL LEVEL BY LEVEL

In [12]:
skills_df['user_level']=skills_df['expected_level'].apply(lambda i: int(i)-1)

### Returning user skill gaps 

In [13]:
## Store gap calculation as new column in df
skills_df['gap']=''
skills_df['gap'] = skills_df.apply(lambda row: int(row.user_level) - int(row.expected_level), axis=1)    

In [14]:
##differentiate between skills that the user already has and those that are lacking
#pull out gaps only, into new df
gaps_df = skills_df.loc[skills_df['gap'] < 0]

## Return list of skill gaps (list as input for Reco/similarity measure)
gaps_df

,Skills,skills_original,expected_level,user_level,gap
0,Applications Development,Applications Development-Level4,4,3,-1
1,Applications Integration,Applications Integration-Level4,4,3,-1
2,Business Environment Analysis,Business Environment Analysis-Level3,3,2,-1
3,Applications Support and Enhancement,Applications Support and Enhancement-Level3,3,2,-1
4,Budgeting,Budgeting-Level3,3,2,-1
5,Business Needs Analysis,Business Needs Analysis-Level3,3,2,-1
6,Business Negotiation,Business Negotiation-Level3,3,2,-1
7,Business Requirements Mapping,Business Requirements Mapping-Level3,3,2,-1
8,Business Risk Management,Business Risk Management-Level3,3,2,-1
9,Change Management,Change Management-Level3,3,2,-1


In [15]:
#end of part 1

## Pull out proficiency description (input for Reco model)

In [16]:
#convert skill gaps into a list as summary for user ##this list not needed in model, will use gaps_df instead  
skillgaps_list = gaps_df['skills_original'].to_list()
skillgaps_list

['Applications Development-Level4',
 'Applications Integration-Level4',
 'Business Environment Analysis-Level3',
 'Applications Support and Enhancement-Level3',
 'Budgeting-Level3',
 'Business Needs Analysis-Level3',
 'Business Negotiation-Level3',
 'Business Requirements Mapping-Level3',
 'Business Risk Management-Level3',
 'Change Management-Level3',
 'Configuration Tracking-Level3',
 'Data Design-Level3',
 'Database Administration-Level4',
 'Emerging Technology Synthesis-Level4',
 'Performance Management-Level4',
 'Problem Management-Level3',
 'Product Management-Level4',
 'Project Management-Level4',
 'Quality Standards-Level4',
 'Software Configuration-Level3',
 'Software Design-Level4',
 'Software Testing-Level3',
 'Solution Architecture-Level4',
 'Stakeholder Management-Level3',
 'System Integration-Level3',
 'Test Planning-Level3',
 'User Interface Design-Level4']

In [18]:
###load proficiency descriptions data from database
skill_prof_data = '/Users/somethingcreative-7/Documents/Tessas Internship/Phase 2 preso/Supplementary documents/proficiency_description.xlsx'
skill_prof_df = pd.read_excel(skill_prof_data)
skill_prof_df=skill_prof_df.drop(columns=['Description'],axis=1)


skill_prof_df['skill_name']= skill_prof_df['Skill-Level']
skill_prof_df['Level']= skill_prof_df['Skill-Level']

##extract skill name 
skill_prof_df['skill_name']= skill_prof_df['skill_name'].apply(lambda i: i.split("-Level",-1)[0])
##extract level as numeric
skill_prof_df['Level']= skill_prof_df['Level'].apply(lambda i: i.rsplit('-Level', 1)[-1])


In [19]:
skill_prof_df.head()

,Skill-Level,clean,skill_name,Level
0,Software Design-Level1,NaN,Software Design,1
1,Software Design-Level2,NaN,Software Design,2
2,Software Design-Level3,Design simple software components assessing f...,Software Design,3
3,Software Design-Level4,Create a software design blueprint based on a...,Software Design,4
4,Software Design-Level5,Translate complex software ideas and concepts...,Software Design,5


In [20]:
skill_prof_df.shape

(804, 4)

## Determine list of skill-proficiency to pull

In [21]:
### for 1 skill- pull out equvalent skill name from proficiency data

gaps_df

,Skills,skills_original,expected_level,user_level,gap
0,Applications Development,Applications Development-Level4,4,3,-1
1,Applications Integration,Applications Integration-Level4,4,3,-1
2,Business Environment Analysis,Business Environment Analysis-Level3,3,2,-1
3,Applications Support and Enhancement,Applications Support and Enhancement-Level3,3,2,-1
4,Budgeting,Budgeting-Level3,3,2,-1
5,Business Needs Analysis,Business Needs Analysis-Level3,3,2,-1
6,Business Negotiation,Business Negotiation-Level3,3,2,-1
7,Business Requirements Mapping,Business Requirements Mapping-Level3,3,2,-1
8,Business Risk Management,Business Risk Management-Level3,3,2,-1
9,Change Management,Change Management-Level3,3,2,-1


To note 1. Each skill gap should return a list of recommendations
///To note 2. [For future iterations] If skill gap is too big (~expected_level - user current level > |2|) then consider splitting the recommendations into 'basic' and 'intermediate' level. this will be operationalised by splitting up the data for analysis, and running them seperately, giving rise to 2 recommendations)
To note 3. from Skills Future CF, not all proficiency levels have a text description(NA), so will have to pick the next best level.

In [22]:
gaps_df['gap']=gaps_df['gap'].apply(int)
gaps_df['gap']=gaps_df['gap'].apply(abs)

In [23]:
###pull out 'expected level' prof description from skill_prof_df db

reco_gap = pd.DataFrame() 

for i in gaps_df['skills_original']:
    row=skill_prof_df.loc[skill_prof_df['Skill-Level']==i]
    reco_gap=reco_gap.append(row) 


In [24]:
###create a col in df that pulls out a list of the prof levels the user needs to pick up (numeric)
gaps_df['all_lvls']= gaps_df['expected_level']
for index, row in gaps_df.iterrows():
    x= list(range(int(row.user_level)+1, int(row.expected_level)+1))
    gaps_df.at[index, 'all_lvls']=x
gaps_df

,Skills,skills_original,expected_level,user_level,gap,all_lvls
0,Applications Development,Applications Development-Level4,4,3,1,[4]
1,Applications Integration,Applications Integration-Level4,4,3,1,[4]
2,Business Environment Analysis,Business Environment Analysis-Level3,3,2,1,[3]
3,Applications Support and Enhancement,Applications Support and Enhancement-Level3,3,2,1,[3]
4,Budgeting,Budgeting-Level3,3,2,1,[3]
5,Business Needs Analysis,Business Needs Analysis-Level3,3,2,1,[3]
6,Business Negotiation,Business Negotiation-Level3,3,2,1,[3]
7,Business Requirements Mapping,Business Requirements Mapping-Level3,3,2,1,[3]
8,Business Risk Management,Business Risk Management-Level3,3,2,1,[3]
9,Change Management,Change Management-Level3,3,2,1,[3]


In [25]:
gaps_df['all_lvls_text']=gaps_df['Skills']

In [26]:
###match skill name and proficiency level across gaps_df and prof desc df
    
for index, row in gaps_df.iterrows():
    if len(row.all_lvls)==1:
        name= row.skills_original
        text= skill_prof_df.loc[skill_prof_df['Skill-Level']==name]['clean'].values[0]
        row.all_lvls_text=str(text)
    else:
        for i in row.all_lvls:        ##for i in list ##one list per row
            name= row.Skills+'-Level'+ str(i)
            #name=row.skills_original
            #find name in skill_prof_df list
            text= skill_prof_df.loc[skill_prof_df['Skill-Level']==name]['clean'].values[0]
            #print(name,text)
            ## append new text to end of all_lvls_text
            row.all_lvls_text = str(row.all_lvls_text) +' '+ str(text)            
    gaps_df.at[index, 'all_lvls_text'] = row.all_lvls_text

In [27]:
gaps_df

,Skills,skills_original,expected_level,user_level,gap,all_lvls,all_lvls_text
0,Applications Development,Applications Development-Level4,4,3,1,[4],Plan the application development process prog...
1,Applications Integration,Applications Integration-Level4,4,3,1,[4],Oversee end to end process of application int...
2,Business Environment Analysis,Business Environment Analysis-Level3,3,2,1,[3],Utilise research instruments quantitative and ...
3,Applications Support and Enhancement,Applications Support and Enhancement-Level3,3,2,1,[3],Analyse application performance statistics an...
4,Budgeting,Budgeting-Level3,3,2,1,[3],Prepare business unit s operational budgets O...
5,Business Needs Analysis,Business Needs Analysis-Level3,3,2,1,[3],Elicit and analyse business requirements from...
6,Business Negotiation,Business Negotiation-Level3,3,2,1,[3],Apply negotiation skills and techniques and d...
7,Business Requirements Mapping,Business Requirements Mapping-Level3,3,2,1,[3],Analyse relevant information from stakeholder...
8,Business Risk Management,Business Risk Management-Level3,3,2,1,[3],Identify risks and their business impact and ...
9,Change Management,Change Management-Level3,3,2,1,[3],Apply change control procedures in work proce...


# 1. Extract proficiency data (skill gaps) for reco model

In [28]:
reco_gap= gaps_df[['Skills','all_lvls_text']]

### text pre-processing for proficiency data

In [29]:
#tokenize proficiency description (after combining across skill levels)
reco_gap['all_lvls_text'] = reco_gap['all_lvls_text'].apply(lambda i: pos_tag(word_tokenize(i),tagset='universal'))

/var/folders/xg/4dfknv856n51lr83nj9crvnh0000gn/T/ipykernel_73040/1830369949.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reco_gap['all_lvls_text'] = reco_gap['all_lvls_text'].apply(lambda i: pos_tag(word_tokenize(i),tagset='universal'))


In [30]:
###lemmatize proficiency description 

##define lem function
wnl = nltk.WordNetLemmatizer()
def lemmaNVAR(wpos):
    lemmas = []
    for w, pos in wpos:
        if pos == 'NOUN':
            lemmas.append(wnl.lemmatize(w.lower(), pos = 'n'))
    return lemmas

reco_gap['all_lvls_text'] = reco_gap['all_lvls_text'].apply(lemmaNVAR)
print(reco_gap['all_lvls_text'])

0     [plan, application, development, process, prog...
1     [oversee, end, process, application, integrati...
2     [utilise, research, instrument, data, informat...
3     [analyse, application, performance, statistic,...
4     [prepare, business, unit, budget, parameter, t...
5     [elicit, analyse, business, requirement, stake...
6     [apply, negotiation, skill, technique, negotia...
7     [analyse, information, stakeholder, map, busin...
8     [identify, risk, business, impact, measure, ri...
9     [apply, change, control, procedure, work, proc...
10    [develop, configuration, management, plan, sys...
11    [identify, data, requirement, design, database...
12    [plan, installation, configuration, upgrading,...
13    [evaluate, technology, trend, need, current, i...
14    [metric, mechanism, network, software, system,...
15    [handle, problem, diagnosis, prioritisation, i...
16    [conceptualise, idea, business, model, prototy...
17    [plan, medium, project, programme, resourc

/var/folders/xg/4dfknv856n51lr83nj9crvnh0000gn/T/ipykernel_73040/1433928153.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reco_gap['all_lvls_text'] = reco_gap['all_lvls_text'].apply(lemmaNVAR)


In [31]:
###identify stopwords
mystopwords=stopwords.words("english") + ['nan'] #remove NAs here

In [32]:
##remove stopwords
reco_gap['all_lvls_text'] = reco_gap['all_lvls_text'].apply(lambda i: [t for t in i if t not in mystopwords])

/var/folders/xg/4dfknv856n51lr83nj9crvnh0000gn/T/ipykernel_73040/1786926765.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reco_gap['all_lvls_text'] = reco_gap['all_lvls_text'].apply(lambda i: [t for t in i if t not in mystopwords])


In [33]:
reco_gap

,Skills,all_lvls_text
0,Applications Development,"[plan, application, development, process, prog..."
1,Applications Integration,"[oversee, end, process, application, integrati..."
2,Business Environment Analysis,"[utilise, research, instrument, data, informat..."
3,Applications Support and Enhancement,"[analyse, application, performance, statistic,..."
4,Budgeting,"[prepare, business, unit, budget, parameter, t..."
5,Business Needs Analysis,"[elicit, analyse, business, requirement, stake..."
6,Business Negotiation,"[apply, negotiation, skill, technique, negotia..."
7,Business Requirements Mapping,"[analyse, information, stakeholder, map, busin..."
8,Business Risk Management,"[identify, risk, business, impact, measure, ri..."
9,Change Management,"[apply, change, control, procedure, work, proc..."


In [34]:
###pre-processing of prof data
##prep df structure as we want to use it as input for reco model
##rename to column headings so it can appended to course data later on
reco_gap.rename(columns = {'Skills': 'course_name'}, inplace = True)
reco_gap.rename(columns = {'all_lvls_text': 'tags'}, inplace = True)
## clean text
reco_gap['tags'] = reco_gap.tags.apply(lambda x: ', '.join([str(i) for i in x]))

reco_gap['tags'] = reco_gap['tags'].str.replace(',',' ')
reco_gap['tags'] = reco_gap['tags'].apply(lambda x:x.lower()) #lower casing the tags column 
##reset index for df
reco_gap=reco_gap.reset_index(level=None, drop=True, inplace=False)

/Users/somethingcreative-7/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/var/folders/xg/4dfknv856n51lr83nj9crvnh0000gn/T/ipykernel_73040/2190446379.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reco_gap['tags'] = reco_gap.tags.apply(lambda x: ', '.join([str(i) for i in x]))
/var/folders/xg/4dfknv856n51lr83nj9crvnh0000gn/T/ipykernel_73040/2190446379.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [35]:
#defining the stemming function to be used later on (see Model Function below)
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

def stem(text):
    y=[]
    
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)

In [36]:
reco_gap

,course_name,tags
0,Applications Development,plan application development process progr...
1,Applications Integration,oversee end process application integratio...
2,Business Environment Analysis,utilise research instrument data informati...
3,Applications Support and Enhancement,analyse application performance statistic ...
4,Budgeting,prepare business unit budget parameter ty...
5,Business Needs Analysis,elicit analyse business requirement stakeh...
6,Business Negotiation,apply negotiation skill technique negotiat...
7,Business Requirements Mapping,analyse information stakeholder map busine...
8,Business Risk Management,identify risk business impact measure ris...
9,Change Management,apply change control procedure work proce...


[future iterations]if gap is 3 or more--> take expected level(x) and extract half the len(total gap) into 'intermediate list' and the remainding (until x== users current level + 1) into 'beginner list' 

# 2. Extract courses data for reco model

In [38]:
coursedata = '/Users/somethingcreative-7/Documents/Tessas Internship/Phase 2 preso/Supplementary documents/coursefeaturesoutput.xlsx'
coursedata = pd.read_excel(coursedata)

In [39]:
new_df_courses=coursedata[['course_name','tags']]
new_df_courses.head()

,course_name,tags
0,Write A Feature Length Screenplay For Film Or ...,write a feature length screenplay for film or ...
1,Business Strategy Business Model Canvas Analys...,business strategy business model canvas analys...
2,Silicon Thin Film Solar Cells,silicon thin film solar cells advanced present...
3,Finance for Managers,finance for managers intermediate number eye f...
4,Retrieve Data using Single-Table SQL Queries,retrieve data using single-table sql queries b...


In [40]:
###tokenising, lemmatising and other processing for course data can be found in the 'Extract_ Course description from Coursera data' script


# Reco based on TF_IDF cosine similarity

### Model function

In [41]:
###vectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
### similarity measure
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances


In [42]:
#cv = CountVectorizer(max_features=5000,stop_words='english') ##normal count vec
cv = TfidfVectorizer(min_df = 2, max_df=0.9,stop_words='english') ##TF-IDF count vec

### for every skill gap (where name and tag already clean/processed), pull out description and run TFIDF and the similarity model to push out 1 recommendation list per skill gap 

In [44]:
###iter over df
for index, row in reco_gap.iterrows():
    model_df=new_df_courses
    target=row['course_name']
    ###combine prof desc row to courses df to create a main df that goes into TF-IDF and then reco model
    combined=model_df.append(row)
    ###applying stemming on the tags column
    combined['tags'] = combined['tags'].apply(stem)  
    ###applying vectorisation on tags
    vectors = cv.fit_transform(combined['tags']).toarray()
    
    ###TF-IDF###
    ##Compute the Cosine Similarity for all courses to job title (last row) based on the count_matrix
    cosine_sim = cosine_similarity(vectors) 
    vec_into_df = pd.DataFrame(cosine_sim[-1],index=combined.course_name,columns=[target])
    #print(vec_into_df.head())
    
    ###sort by most similar/ closest datapoint to target job title
    recommended_courses= vec_into_df[target].sort_values(ascending=False)[:6]

    #print(recommended_courses)

    recommended_courses=list(recommended_courses.index.values)
    print((index+1),'.','For the competency', recommended_courses[0], ': the top 5 most relevant courses are:', recommended_courses[1:])
    

1 . For the competency Applications Development : the top 5 most relevant courses are: ['Web Application Security Testing with Burp Suite', 'Systems and Application Security', 'Web Application Security Testing with OWASP ZAP', 'Software Security', "Intermediate Intel' Distribution of OpenVINO' toolkit for Deep Learning Applications"]
2 . For the competency Applications Integration : the top 5 most relevant courses are: ['Continuous Integration', 'Web Application Development Basic Concepts', 'Build a Firebase Android Application (Part II)', 'Getting Started With Application Development', 'Web Application Security Testing with Burp Suite']
3 . For the competency Business Environment Analysis : the top 5 most relevant courses are: ['Business intelligence and data analytics Generate insights', 'Strategic management Be competitive', 'Organisational design Know your organisation', 'Foundations of Business Strategy', 'Survey analysis to Gain Marketing Insights']


KeyboardInterrupt: 

In [45]:
def bullet(x,reco):
    print(x. reco)

In [57]:
for i in recommended_courses[1:]:
    print(i)
    #print(str(index(i)), str(i))
        

Business intelligence and data analytics Generate insights
Strategic management Be competitive
Organisational design Know your organisation
Foundations of Business Strategy
Survey analysis to Gain Marketing Insights


In [53]:
#end of model

# Pull out course details for each recommendation 

In [81]:
#create df for output
course_details = pd.DataFrame(columns=['Course Name','University','Difficulty Level','Course Rating','Course URL','Course Description','Skills'])
course_details.head()

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills


In [82]:
courseradb = pd.read_csv('/Users/somethingcreative-7/Documents/Tessas Internship/code/coursera.csv') #only used to pull out course details for user at the end


In [83]:
recommended_courses=['Applications Development','Web Application Security Testing with Burp Suite', 'Systems and Application Security', 'Web Application Security Testing with OWASP ZAP', 'Coding for Designers Managers & Entrepreneurs III', 'Identifying Security Vulnerabilities in C/C++Programming', "Intermediate Intel' Distribution of OpenVINO' toolkit for Deep Learning Applications", 'Software Security', 'Getting Started With Application Development', 'Exploiting and Securing Vulnerabilities in Java Applications', 'Web Connectivity and Security in Embedded Systems']

In [84]:
for i in recommended_courses[1:]:
    row=courseradb.loc[courseradb['Course Name']==i]
    course_details=course_details.append(row) 

In [85]:
course_details.index = np.arange(1, len(course_details) + 1) #reset df index to start from 1
course_details

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills
1,Web Application Security Testing with Burp Suite,Coursera Project Network,Intermediate,3.9,https://www.coursera.org/learn/web-application...,"By the end of this project, you will learn the...",Software Security burp suite hypertext trans...
2,Systems and Application Security,(ISC)',Advanced,4.8,https://www.coursera.org/learn/systems-applica...,Welcome to Systems and Application Security Co...,virtual machine encryption process privacy ...
3,Web Application Security Testing with OWASP ZAP,Coursera Project Network,Beginner,4.2,https://www.coursera.org/learn/web-application...,"By the end of this project, you will learn the...",owasp zap uniform resource locator penetrati...
4,Identifying Security Vulnerabilities in C/C++P...,"University of California, Davis",Advanced,4.6,https://www.coursera.org/learn/identifying-sec...,This course builds upon the skills and coding ...,verification and validation c++ setuid race...
5,Intermediate Intel' Distribution of OpenVINO' ...,Intel,Advanced,5,https://www.coursera.org/learn/int-openvino,This course is designed for application develo...,mobile application testing learning tools int...
6,Software Security,"University of Maryland, College Park",Advanced,4.6,https://www.coursera.org/learn/software-security,This course we will explore the foundations of...,internet security SQL security Security Sof...
7,Getting Started With Application Development,Google Cloud,Beginner,4.5,https://www.coursera.org/learn/getting-started...,"In this course, application developers learn h...",Google Cloud Platform Cloud Platforms Cloud ...
8,Exploiting and Securing Vulnerabilities in Jav...,"University of California, Davis",Beginner,4.2,https://www.coursera.org/learn/exploiting-secu...,"In this course, we will wear many hats. With o...",text editors viewer spring framework source...
9,Web Connectivity and Security in Embedded Systems,EIT Digital,Advanced,4.5,https://www.coursera.org/learn/iot-connectivit...,Welcome to Web Connectivity and Security in Cy...,attack model cryptographic protocol applicat...


#end

In [1176]:
#listA = gaps_df.loc[gaps_df['gap']<3]
#listA

In [1177]:
#listB = gaps_df.loc[gaps_df['gap']>2]
